In [ ]:
import yfinance as yf
import pandas as pd

# Fetch historical data for NVIDIA
ticker = 'NVDA'
start_date = '2024-01-01'
end_date = '2025-03-23'
data = yf.download(ticker, start=start_date, end=end_date)

# Display the first few rows
print(data.tail())



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            NVDA        NVDA        NVDA        NVDA       NVDA
Date                                                                 
2025-03-17  119.529999  122.889999  118.029999  122.739998  255501500
2025-03-18  115.430000  119.019997  114.540001  118.000000  299686900
2025-03-19  117.519997  120.449997  115.680000  117.269997  273426200
2025-03-20  118.529999  120.199997  116.470001  116.550003  248829700
2025-03-21  117.699997  117.989998  115.419998  116.940002  265954100


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA

# Load historical NVDA stock data
def get_historical_data():
    stock_data = yf.download('NVDA', start='2020-01-01', end='2025-03-07')
    return stock_data['Close']

# Train ARIMA model and predict next day's price
def predict_next_day_price(stock_data):
    model = ARIMA(stock_data, order=(5,1,0))  # Example parameters
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=5)
    return forecast

# Simulate trading strategy
def trading_agent(predictions, balance, shares):
    decision = "HOLD"
    if predictions.iloc[0] > predictions.iloc[-1]:
        decision = "SELL"
    elif predictions.iloc[0] < predictions.iloc[-1]:
        decision = "BUY"
    return decision

# Running the simulation
stock_prices = get_historical_data()
predictions = predict_next_day_price(stock_prices)
decision = trading_agent(predictions, 10000, 0)
print("Trading Decision for Next day:", decision)

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Trading Decision for Next day: BUY


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA

# Load historical NVDA stock data till today
def get_historical_data():
    stock_data = yf.download('NVDA', start='2020-01-01', end='2025-03-07')  # End date as today (T-1)
    return stock_data['Close']

# Train ARIMA model and predict the next 5 days
def predict_next_days(stock_data):
    model = ARIMA(stock_data, order=(5,1,0))  # Example (p,d,q) parameters
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=5)  # Predicting next 5 days
    return forecast

# Simulate trading strategy based on predictions
def trading_agent(predictions, balance, shares, stock_price):
    decisions = []
    for i in range(len(predictions)):
        decision = "HOLD"
        if i == 0:
            # First day: Buy as many shares as possible
            shares_to_buy = balance // stock_price
            balance -= shares_to_buy * stock_price
            decision = f"BUY {shares_to_buy} shares"
        else:
            if predictions.iloc[i] > predictions.iloc[i-1]:  # Price increase, hold or buy
                decision = "HOLD"
            else:  # Price drop, sell
                decision = f"SELL {shares} shares"
                balance += shares * predictions.iloc[i]
                shares = 0  # Sold all shares
        decisions.append(decision)
    return decisions, balance, shares

# Running the simulation
stock_prices = get_historical_data()
predictions = predict_next_days(stock_prices)

# Assume stock price on first day of prediction is the last actual closing price
initial_stock_price = stock_prices.iloc[-1]

# Start with initial conditions
initial_balance = 10000
initial_shares = 0

# Execute trading strategy
decisions, final_balance, final_shares = trading_agent(predictions, initial_balance, initial_shares, initial_stock_price)

# Prepare results for display
results_df = pd.DataFrame({
    "Date": pd.date_range(start="2025-03-10", periods=5, freq='B'),
    "Predicted Price": predictions.values,
    "Decision": decisions
})

print("Trading decision for five days",results_df)

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Trading decision for five days         Date  Predicted Price  \
0 2025-03-10       111.020530   
1 2025-03-11       110.832228   
2 2025-03-12       111.197960   
3 2025-03-13       111.127217   
4 2025-03-14       111.490874   

                                            Decision  
0  BUY Ticker\nNVDA    90.0\nName: 2025-03-06 00:...  
1                                      SELL 0 shares  
2                                               HOLD  
3                                      SELL 0 shares  
4                                               HOLD  


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
